In [1]:
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import glob
from os.path import join
%matplotlib inline

In [2]:
from src.params import ROOT_DIR, PROCESSED
print(ROOT_DIR)
PROCESSED

Project Directory: /data2/isshamie/time_neutrophils

Config paths:
__name__ src.params
__package__ src
__file__ /data2/isshamie/time_neutrophils/src/params.py
__cached__ /data2/isshamie/time_neutrophils/src/__pycache__/params.cpython-37.pyc
path /data2/isshamie/time_neutrophils/src/params.py
DATA_DIR /data2/isshamie/time_neutrophils/data
RAW_DIR /data2/isshamie/time_neutrophils/data/raw
PROCESSED /data2/isshamie/time_neutrophils/data/processed
PARAM_DIR /data2/isshamie/time_neutrophils/parameters
RESULTS /data2/isshamie/time_neutrophils/results
FIGURES_DIR /data2/isshamie/time_neutrophils/figures
/data2/isshamie/time_neutrophils


'/data2/isshamie/time_neutrophils/data/processed'

In [11]:
fig_dir = join(ROOT_DIR,"figures/umap_out/2_fix/")
savedir = join(ROOT_DIR,"results/umap_out/2_fix/")

In [4]:
meta_df = pd.read_csv(join(PROCESSED,"meta.tsv"),sep="\t", index_col=0)
meta_df

,Timepoint,Stimuli,Sample,Genotype,Stimuli Names
0_2G3,3,G,2,WT,Nec1s
1_2G3,3,G,2,WT,Nec1s
2_2G3,3,G,2,WT,Nec1s
3_2G3,3,G,2,WT,Nec1s
4_2G3,3,G,2,WT,Nec1s
...,...,...,...,...,...
4592_4B46,46,B,4,WT,zVD
4593_4B46,46,B,4,WT,zVD
4594_4B46,46,B,4,WT,zVD
4595_4B46,46,B,4,WT,zVD


In [6]:
raw = pickle.load(open(join(PROCESSED,"data_df_z.p"),"rb"))
raw

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity
0_2G3,62.466201,-9.764209,2.257279,-0.613713,-0.912228,-1.053166
1_2G3,-0.327282,-9.343066,84.071640,-0.941838,-0.915353,-1.213429
2_2G3,-0.590511,-7.180160,15.145722,-0.891318,-0.914065,-1.198231
3_2G3,-0.531419,-8.459477,30.618888,-0.818131,-0.911537,-1.199510
4_2G3,-0.644231,0.652443,1.079209,-0.827969,-0.913367,-1.194802
...,...,...,...,...,...,...
4592_4B46,0.505380,0.034244,-0.258939,-1.286265,-0.804490,0.235745
4593_4B46,1.332670,-4.715006,2.515822,-1.462160,-0.665997,0.404229
4594_4B46,0.188431,-0.785514,0.351688,-1.305067,-0.060241,0.186008
4595_4B46,-0.552907,0.652443,0.056995,-0.659327,2.145113,0.165367


In [14]:
for curr_param in glob.glob(join(ROOT_DIR,"results/umap_out/2/*p")):
    neigh = curr_param.strip(".p").split("_")[-2]
    dist = curr_param.strip(".p").split("_")[-1]
    if int(neigh) == 100 and (dist) == "0":
        print(curr_param)
        inp = pickle.load(open(curr_param,"rb"))
        if not len(inp) == 2:
            continue
        embedding,samples_inds = inp[0], inp[1]
        curr_df = meta_df.loc[samples_inds]
        curr_df["embedding_1"] = embedding[:,0]
        curr_df["embedding_2"] = embedding[:,1]
        curr_df = pd.merge(curr_df, raw,how='inner',left_index=True, right_index=True)

        # Create figures
        color_labels = curr_df["Stimuli Names"].unique()
        rgb_values = (sns.color_palette("Set2", len(color_labels)))
        color_map = dict(zip(color_labels, rgb_values))

        plt.figure(figsize=(15,15))
        pallete=sns.color_palette("bright", len(color_labels))
        sns.scatterplot(data=curr_df,x="embedding_1", y="embedding_2", palette=pallete, hue='Stimuli Names',style="Genotype",
                                size=0.5)

        plt.legend(loc='upper right')


        title = f"Neighbors={neigh}\nMinimum Distance={dist}\nN Samples = {len(embedding)}"
        plt.title(title)
        plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_umap_stimuli.png"))
        plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_umap_stimuli.pdf"))
        plt.close()

        for i in raw.columns.values:
            plt.figure(figsize=(15,15))
            pallete=sns.color_palette("bright", len(color_labels))
            sns.scatterplot(data=curr_df,x="embedding_1", y="embedding_2", hue=i,
                            style="Genotype", size=0.5)
            #curr_df.plot.scatter("embedding_1","embedding_2", hue=curr_df['Stimuli'])#.map(color_map),s=0.5)
            plt.legend(loc='upper right')
            plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i.replace(' ','')}.png"))
            plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i.replace(' ','')}.pdf"))
            plt.close()

/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_0_100_0.p
/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_1_100_0.p
/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_2_100_0.p


In [15]:
for curr_param in glob.glob(join(ROOT_DIR,"results/umap_out/2/*p")):
    neigh = curr_param.strip(".p").split("_")[-2]
    dist = curr_param.strip(".p").split("_")[-1]
    if (int(neigh) == 500 or int(neigh) == 15)  and (dist) == "0":
        print(curr_param)
        inp = pickle.load(open(curr_param,"rb"))
        if not len(inp) == 2:
            continue
        embedding,samples_inds = inp[0], inp[1]
        curr_df = meta_df.loc[samples_inds]
        curr_df["embedding_1"] = embedding[:,0]
        curr_df["embedding_2"] = embedding[:,1]
        curr_df = pd.merge(curr_df, raw,how='inner',left_index=True, right_index=True)

        # Create figures
        color_labels = curr_df["Stimuli Names"].unique()
        rgb_values = (sns.color_palette("Set2", len(color_labels)))
        color_map = dict(zip(color_labels, rgb_values))

        plt.figure(figsize=(15,15))
        pallete=sns.color_palette("bright", len(color_labels))
        sns.scatterplot(data=curr_df,x="embedding_1", y="embedding_2", palette=pallete, hue='Stimuli Names',style="Genotype",
                                size=0.5)

        plt.legend(loc='upper right')


        title = f"Neighbors={neigh}\nMinimum Distance={dist}\nN Samples = {len(embedding)}"
        plt.title(title)
        plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_umap_stimuli.png"))
        plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_umap_stimuli.pdf"))
        plt.close()

        for i in raw.columns.values:
            plt.figure(figsize=(15,15))
            pallete=sns.color_palette("bright", len(color_labels))
            sns.scatterplot(data=curr_df,x="embedding_1", y="embedding_2", hue=i,
                            style="Genotype", size=0.5)
            #curr_df.plot.scatter("embedding_1","embedding_2", hue=curr_df['Stimuli'])#.map(color_map),s=0.5)
            plt.legend(loc='upper right')
            plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i.replace(' ','')}.png"))
            plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i.replace(' ','')}.pdf"))
            plt.close()

/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_2_500_0.p
/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_0_15_0.p
/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_1_500_0.p
/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_2_15_0.p
/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_0_500_0.p
/data2/isshamie/time_neutrophils/results/umap_out/2/embedding_1_15_0.p


In [8]:
n_neighbors_l = [100]
min_distance_l = [0]
n_subsample = 100
n_iter = 3

# for i_ter in range(n_iter):
#     print('i_ter',i_ter)
#     for neigh in n_neighbors_l:
#         print('number of neighbors', neigh)
#         for dist in min_distance_l:
#             print('minimum distance', dist)
#             # File name
#             curr_param = f"{savedir}/embedding_{i_ter}_{neigh}_{dist}.p" #umap_results
#             print(curr_param)

#             inp = pickle.load(open(curr_param,"rb"))
#             if not len(inp) == 2:
#                 continue
#             embedding,samples_inds = inp[0], inp[1]
#             curr_df = meta_df.loc[samples_inds]
#             curr_df["embedding_1"] = embedding[:,0]
#             curr_df["embedding_2"] = embedding[:,1]
#             curr_df = pd.merge(curr_df, raw,how='inner',left_index=True, right_index=True)

#             # Create figures
#             color_labels = curr_df["Stimuli Names"].unique()
#             rgb_values = (sns.color_palette("Set2", len(color_labels)))
#             color_map = dict(zip(color_labels, rgb_values))

#             plt.figure(figsize=(15,15))
#             pallete=sns.color_palette("bright", len(color_labels))
#             sns.scatterplot(data=curr_df,x="embedding_1", y="embedding_2", palette=pallete, hue='Stimuli Names',style="Genotype",
#                                     size=0.5)

#             plt.legend(loc='upper right')


#             title = f"Neighbors={neigh}\nMinimum Distance={dist}\nN Samples = {len(embedding)}"

#             neigh = curr_param.strip(".p").split("_")[-2]
#             dist = curr_param.strip(".p").split("_")[-1]
#             plt.title(title)
#             plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i_ter}_umap_stimuli.png"))
#             plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i_ter}_umap_stimuli.pdf"))
#             plt.close()

#             for i in raw.columns.values:
#                 plt.figure(figsize=(15,15))
#                 pallete=sns.color_palette("bright", len(color_labels))
#                 sns.scatterplot(data=curr_df,x="embedding_1", y="embedding_2", hue=i,
#                                 style="Genotype", size=0.5)
#                 #curr_df.plot.scatter("embedding_1","embedding_2", hue=curr_df['Stimuli'])#.map(color_map),s=0.5)
#                 plt.legend(loc='upper right')
#                 plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i.replace(' ','')}.png"))
#                 plt.savefig(os.path.join(fig_dir,f"{neigh}_{dist}_{i.replace(' ','')}.pdf"))
#                 plt.close()

i_ter 0
number of neighbors 500
minimum distance 0.1
/data2/isshamie/time_neutrophils/umap_out/2_fix//embedding_0_500_0.1.p


FileNotFoundError: [Errno 2] No such file or directory: '/data2/isshamie/time_neutrophils/umap_out/2_fix//embedding_0_500_0.1.p'